In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
# Any results you write to the current directory are saved as output.

/kaggle/input/ifood-2019-fgvc6/train_labels.csv
/kaggle/input/ifood-2019-fgvc6/test_set.zip
/kaggle/input/ifood-2019-fgvc6/val_labels.csv
/kaggle/input/ifood-2019-fgvc6/val_set.zip
/kaggle/input/ifood-2019-fgvc6/sample_submission.csv
/kaggle/input/ifood-2019-fgvc6/ifood2019_sample_submission.csv
/kaggle/input/ifood-2019-fgvc6/train_set.zip
/kaggle/input/ifood-2019-fgvc6/class_list.txt


In [2]:
from zipfile import ZipFile
import shutil
import os

if not os.path.exists("./train_set"):
    for file_name in ['train_set.zip', 'val_set.zip', 'test_set.zip']:
        with ZipFile('../input/ifood-2019-fgvc6/' + file_name, 'r') as zipObj:
            print("unzipping", file_name)
            zipObj.extractall('./')

for dirname, _, _ in os.walk('./'):
    print(dirname)

unzipping train_set.zip
unzipping val_set.zip
unzipping test_set.zip
./
./train_set
./test_set
./val_set


In [3]:
df_train = pd.read_csv('../input/ifood-2019-fgvc6/train_labels.csv')
df_val   = pd.read_csv('../input/ifood-2019-fgvc6/val_labels.csv')
df_test = pd.read_csv('../input/ifood-2019-fgvc6/sample_submission.csv')

df_train['label'] = df_train['label'].astype(str)
df_val['label'] = df_val['label'].astype(str)

train_size = df_train.shape[0]
val_size = df_val.shape[0]
test_size = df_test.shape[0]
num_classes = df_train['label'].nunique()

In [4]:
for ind, item in df_train.iterrows():
    if not os.path.exists("./train_set/" + item['label']):
        os.mkdir("./train_set/" + item['label'])
    os.rename("./train_set/"+item['img_name'], "./train_set/"+item['label']+"/"+item['img_name'])
    
for ind, item in df_val.iterrows():
    if not os.path.exists("./val_set/" + item['label']):
        os.mkdir("./val_set/" + item['label'])
    os.rename("./val_set/"+item['img_name'], "./val_set/"+item['label']+"/"+item['img_name'])

os.mkdir("./test_set/0")
for ind, item in df_test.iterrows():
    os.rename("./test_set/"+item['img_name'], "./test_set/0/"+item['img_name'])

In [5]:
import torch
import torchvision
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import sampler

NUM_TRAIN = 10000
NUM_VAL = 1000
BATCH_SIZE = 64

transform_train = T.Compose([
    T.RandomResizedCrop(224),
    T.RandomHorizontalFlip(),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_test = T.Compose([
    T.Resize(256),
    T.CenterCrop(224),
    T.ToTensor(),
    T.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

data_train = torchvision.datasets.ImageFolder(
                root="./train_set",
                transform=transform_train
            )
# loader_train = DataLoader(data_train, batch_size=BATCH_SIZE, 
#                           sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))
loader_train = DataLoader(data_train, batch_size=BATCH_SIZE, shuffle=True)
data_val = torchvision.datasets.ImageFolder(
                root="./val_set",
                transform=transform_test
            )
# loader_val = DataLoader(data_val, batch_size=BATCH_SIZE,
#                        sampler=sampler.SubsetRandomSampler(range(NUM_VAL)))
loader_val = DataLoader(data_val, batch_size=BATCH_SIZE, shuffle=False)
data_test = torchvision.datasets.ImageFolder(
                root="./test_set",
                transform=transform_test
            )
loader_test = DataLoader(data_test, batch_size=BATCH_SIZE, shuffle=False)

In [6]:
images, labels = next(iter(loader_train))
print("batch data shape: ", images.shape)
print("batch label shape: ", labels.shape)

batch data shape:  torch.Size([64, 3, 224, 224])
batch label shape:  torch.Size([64])


In [7]:
USE_GPU = True

dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print('using device:', device)

using device: cuda


In [8]:
def train(model, optimizer, epochs=1, print_every=100):
    global best_acc
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    
    for e in range(epochs):
        print('Epoch %d' % e)
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if t % print_every == 0:
                print('Iteration %d, loss = %.4f' % (t, loss.item()))
                train_acc = evaluate(loader_train, model, "train")
                val_acc = evaluate(loader_val, model, "val")
        print()
        
#         save_checkpoint({
#                 'epoch': e + 1,
#                 'state_dict': model.state_dict(),
#                 'best_acc': best_acc,
#             }, is_best)
        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), 'model_best.pth.tar')

In [9]:
def evaluate(loader, model, mode):
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
#             _, preds = scores.max(1)
            # top-3 prediction
            _, preds = torch.topk(scores, k=3, dim=1)
            num_correct += (preds == y.view(y.shape[0],1)).sum()
            num_samples += preds.size(0)

        acc = float(num_correct) / num_samples
        print('%s: %d / %d correct (%.2f)' % (mode, num_correct, num_samples, 100 * acc))
        return acc

In [10]:
def predict(loader, model):
    model.eval()
    y_pred = torch.tensor([], dtype=torch.long, device=device)

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
#             _, preds = scores.max(1)
            # top-3 prediction
            _, preds = torch.topk(scores, k=3, dim=1)
            y_pred = torch.cat((y_pred, preds), 0)
        return y_pred

In [11]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [12]:
learning_rate = 1e-3

model = nn.Sequential(
    nn.Conv2d(3, 32, (5, 5), padding=2),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.Conv2d(32, 32, (5, 5), padding=2),
    nn.BatchNorm2d(32),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(0.25),
    
    nn.Conv2d(32, 64, (3, 3), padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.Conv2d(64, 64, (3, 3), padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),
    nn.Dropout(0.25),
    
    nn.Flatten(),
    nn.Linear(64 * 56 * 56, 512),
    nn.BatchNorm1d(512),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(512, num_classes),
)

# optimizer = optim.SGD(model.parameters(), lr=learning_rate,
#                      momentum=0.9, nesterov=True)
optimizer = optim.Adam(model.parameters())

In [13]:
best_acc = 0
train(model, optimizer, epochs=1, print_every=100)

Epoch 0
Iteration 0, loss = 5.5800
train: 1451 / 118475 correct (1.22)
val: 132 / 11994 correct (1.10)
Iteration 100, loss = 5.5933
train: 3171 / 118475 correct (2.68)
val: 267 / 11994 correct (2.23)
Iteration 200, loss = 5.3725
train: 6658 / 118475 correct (5.62)
val: 694 / 11994 correct (5.79)
Iteration 300, loss = 5.4536
train: 7734 / 118475 correct (6.53)
val: 819 / 11994 correct (6.83)
Iteration 400, loss = 5.1450
train: 8990 / 118475 correct (7.59)
val: 964 / 11994 correct (8.04)
Iteration 500, loss = 5.4150
train: 9807 / 118475 correct (8.28)
val: 1105 / 11994 correct (9.21)
Iteration 600, loss = 5.1616
train: 10658 / 118475 correct (9.00)
val: 1238 / 11994 correct (10.32)
Iteration 700, loss = 5.1332
train: 11612 / 118475 correct (9.80)
val: 1343 / 11994 correct (11.20)
Iteration 800, loss = 5.1191
train: 11997 / 118475 correct (10.13)
val: 1409 / 11994 correct (11.75)
Iteration 900, loss = 5.0143
train: 12599 / 118475 correct (10.63)
val: 1591 / 11994 correct (13.26)
Iteration

In [14]:
y_pred = predict(loader_test, model)
df_test['label'] = [' '.join(item) for item in y_pred.cpu().numpy().astype(str)]
df_test.to_csv("submission.csv", index=False)

In [15]:
for path in ["./train_set", "./val_set", "./test_set"]:
    shutil.rmtree(path)